# GP

What MSE does the GP achieve

In [1]:
import torch

import gpytorch
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel
from gpytorch.distributions import MultivariateNormal

from os.path import join

import sys
sys.path.append('../')
import utils

In [2]:
assert torch.cuda.is_available(), 'CUDA is not available.'

## Load Data

In [3]:
fname = join('/home/squirt/Documents/data/weather_data/', 'all_data.h5')

In [4]:
split = 0.5
train_data, test_data = utils.get_data(fname, split)

In [5]:
def combine_data(data_tuple:tuple[torch.tensor]) -> tuple[torch.tensor]:
    '''
    Add Landmass to x data. Return x,y tensors
    Input:
        - data_tuple (tuple[torch.tensor]): tuple of landmass,x,y tensors
    '''
    l,x,y = data_tuple

    # Combine
    l = l.unsqueeze(1)
    x = torch.cat((l, x), 1)
    x = x.contiguous()
    y = y.contiguous()
    return (x,y)

In [6]:
train_data = combine_data(train_data)
test_data = combine_data(test_data)

In [7]:
x,y = train_data
x = x.view(-1, 71*3*2*2)
y = y.view(-1, 70*2*2*2)

#x = x.cuda()
#y = y.cuda()

In [8]:
x_test, y_test = test_data
x_test = x_test.view(-1, 71*3*2*2)
y_test = y_test.view(-1, 70*2*2*2)

#x_test = x_test.cuda()
#y_test = y_test.cuda()

## Define GP

Define Model using gptorch

In [9]:
'''
class GaussianModel(gpytorch.models.ExactGP):
    def __init__(self, likelihood):
        # Initialize with None data - data will be set later
        super(GaussianModel, self).__init__(None, None, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.RBFKernel()

    def set_train_data(self, train_x, train_y):
        self.train_inputs = (train_x,) 
        self.train_targets = train_y 

    def forward(self, x, ):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
'''
class MultitaskGaussianModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGaussianModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([train_y.shape[1]]))
        self.covar_module = gpytorch.kernels.RBFKernel(batch_shape=torch.Size([train_y.shape[1]]))
        self.task_covar_module = gpytorch.kernels.IndexKernel(num_tasks=train_y.shape[1], rank=1)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        task_indices = torch.arange(self.train_targets.shape[-1]).unsqueeze(0).repeat(x.shape[0], 1)
        covar_task = self.task_covar_module(task_indices)
        covar_x = covar_x.mul(covar_task)
        return gpytorch.distributions.MultitaskMultivariateNormal.from_batch_shape(mean_x, covar_x)




In [10]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=y.shape[1])
#likelihood = likelihood.cuda()

model = MultitaskGaussianModel(x, y, likelihood=likelihood)
#model = model.cuda()

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
#mll = mll.cuda()

## Train Model

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [12]:
# Training loop
model.set_train_data(x, y)
likelihood.train()

for epoch in range(10):
    # Train Model
    model.train()
    optimizer.zero_grad()
        
    # This is a key part: call your model on a batch
    output = model(x)
    print(output.shape)
    loss = -mll(output, y)
    loss.backward()
        
    optimizer.step()
    print('Here')

    #Test Model
    model.eval()
    loss = torch.nn.functional.mse_loss(model(x_test).mean, y_test)
    print(f'Epoch {epoch} - Loss: {loss}')

: 

In [ ]:
y.shape
